In [136]:
import os
import pandas as pd
import numpy as np
import json
import datetime

import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import chart_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [206]:
def get_extra_datetime_cols(df, datecol, date_format="%Y-%m-%d"):
    df['datetime'] = [datetime.datetime.strptime(x, date_format) for x in df[datecol]]
#     df['year'] = [datetime.date(x.year, 1, 1) for x in df['datetime']]
#     df['month'] = [datetime.date(x.year, x.month, 1) for x in df['datetime']]
#     df['week'] = [(x - datetime.timedelta(days=x.weekday())) for x in
#                           df['datetime']]
#     df['day'] = df['date']
    df['day_string'] = [datetime.datetime.strftime(x, date_format)[:10] for x in df['datetime']]
    return df

In [207]:
with open("/Users//Downloads/exchange-balance-stacked.json") as f:
    data = json.load(f)

# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
# print(data)

In [208]:
df = pd.DataFrame()
for row in data:
    out = row.get('o').copy()
    out['date'] = row.get('t')
    df = df.append(out, ignore_index=True)
cols = list(df.columns.values)

In [209]:
df = get_extra_datetime_cols(df, 'date', "%Y-%m-%dT00:00:00Z")
df

,binance,bitfinex,bitmex,bitstamp,bittrex,coinbase,coincheck,date,gemini,hitbtc,huobi,kraken,luno,okex,poloniex,datetime,day_string
0,242493.500888,217083.167790,244107.875077,167055.750209,123500.137641,983142.974613,32867.186858,2019-12-31T00:00:00Z,63533.020759,77715.936925,350413.873876,147365.829262,17703.571301,171725.281927,62237.250484,2019-12-31,2019-12-31
1,243012.589271,217560.277252,244868.905909,167415.610206,123519.564700,983535.091648,32910.729833,2020-01-01T00:00:00Z,62977.930521,77432.130632,350985.548217,147680.767835,17696.907500,171810.747259,62314.993111,2020-01-01,2020-01-01
2,242908.463240,218091.560981,246353.521046,166939.197732,123201.467454,983469.681663,32891.834145,2020-01-02T00:00:00Z,61606.489345,77418.028137,350885.083843,148236.075201,17818.843808,172138.170150,62232.526277,2020-01-02,2020-01-02
3,243292.545828,220921.524959,245964.441069,166342.373885,122780.051440,983768.565316,32933.582634,2020-01-03T00:00:00Z,62881.014592,77928.679298,350424.741949,147270.469737,17897.979141,172858.879730,62028.468052,2020-01-03,2020-01-03
4,243547.048084,222187.378366,246001.078915,168150.459710,122869.957147,983572.386777,32904.589670,2020-01-04T00:00:00Z,63090.434676,77925.504356,350326.433010,148154.207989,17839.053660,173632.216233,61915.075347,2020-01-04,2020-01-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,279885.865629,74966.645217,104646.917709,124747.910198,98765.244200,908533.196538,34719.503086,2020-10-17T00:00:00Z,120711.897624,67461.019731,279353.345924,147093.496855,17342.175909,202573.384616,47748.261194,2020-10-17,2020-10-17
292,279948.919571,74907.268335,104337.439834,124836.647760,98631.419835,908909.532304,34720.160689,2020-10-18T00:00:00Z,120626.672012,67475.522227,276432.980244,146480.358274,17331.968260,202691.166996,47722.921152,2020-10-18,2020-10-18
293,278220.183885,75285.279012,104575.943800,125166.909673,98489.309676,909304.387448,34745.592974,2020-10-19T00:00:00Z,120384.013557,67642.698192,273597.808701,146341.657872,17205.334316,202732.640982,47641.768342,2020-10-19,2020-10-19
294,278441.873444,75902.975030,104857.526045,124492.984298,98339.687886,908601.612035,34507.112098,2020-10-20T00:00:00Z,120783.364575,67483.464136,271735.921582,146780.980780,17018.352316,203254.436168,47569.986153,2020-10-20,2020-10-20


In [210]:
pct_cols = []
delta_cols = []
for col in cols:
    if col != 'date':
        df['{}_pct_change'.format(col)] = (df[col] / df[col][0]) - 1
        pct_cols.append('{}_pct_change'.format(col))
        
        df['{}_net_change'.format(col)] = df[col] - df[col][0]
        delta_cols.append('{}_net_change'.format(col))

In [211]:
cm = pd.read_csv("btc.csv",
                   usecols=['date', 'PriceUSD']).dropna()
cm = get_extra_datetime_cols(cm, 'date')

In [212]:
chart_utils.single_axis_chart(
    df.loc[df['date'] >= '2020-01-01'], 'date', pct_cols,
    y1_series_title='Exchange Balances % Change 2020 YTD', data_source='Glassnode',
    y1_series_axis_format=',.2%')

In [213]:
chart_utils.single_axis_chart(
    df.loc[df['date'] >= '2020-01-01'], 'date', delta_cols,
    y1_series_title='Exchange Balances Net Change 2020 YTD', data_source='Glassnode')

In [257]:
bitfinex = pd.read_csv("/Users/tylerwood/Downloads/bitfinex-trading-volume-1-year.csv")

In [258]:
bitfinex['date'] = pd.to_datetime(bitfinex['snapped_at'])

In [259]:
bitfinex

,snapped_at,volume,date
0,2019-10-24 14:00:00 +0000,1.356168e+08,2019-10-24 14:00:00+00:00
1,2019-10-25 14:00:00 +0000,7.429731e+07,2019-10-25 14:00:00+00:00
2,2019-10-26 14:00:00 +0000,6.345130e+08,2019-10-26 14:00:00+00:00
3,2019-10-27 14:00:00 +0000,1.645554e+08,2019-10-27 14:00:00+00:00
4,2019-10-28 14:00:00 +0000,2.096511e+08,2019-10-28 14:00:00+00:00
...,...,...,...
361,2020-10-19 14:00:00 +0000,7.025872e+07,2020-10-19 14:00:00+00:00
362,2020-10-20 14:00:00 +0000,1.519661e+08,2020-10-20 14:00:00+00:00
363,2020-10-21 14:00:00 +0000,2.241203e+08,2020-10-21 14:00:00+00:00
364,2020-10-22 14:00:00 +0000,3.943222e+08,2020-10-22 14:00:00+00:00


In [260]:
bitfinex = get_extra_datetime_cols(bitfinex, 'snapped_at', "%Y-%m-%d 14:00:00 +0000")
bitfinex = bitfinex.merge(cm, on='day_string')

In [262]:
bitfinex['btc_volume'] = bitfinex['volume'] / bitfinex['PriceUSD']
bitfinex['btc_volume_rolling_28d'] = bitfinex.btc_volume.rolling(28, min_periods=28).mean()
bitfinex['btc_volume_rolling_7d'] = bitfinex.btc_volume.rolling(7, min_periods=7).mean()

In [263]:
bitfinex = bitfinex.merge(df, on='day_string')

In [264]:
chart_utils.two_axis_chart(
    bitfinex.loc[bitfinex['date_x'] >= '2020-01-01'], 'date_x', 'btc_volume_rolling_28d',
    y2_series='PriceUSD',
    y1_series_title='Bitfinex Exchange Normalized Trade Volume (BTC)', data_source='CoinGecko')

In [267]:
chart_utils.two_axis_chart(
    bitfinex.loc[bitfinex['date_x'] >= '2020-01-01'], 'date_x',
    y1_series='bitfinex_pct_change', y2_series='btc_volume_rolling_28d', y1_series_axis_format=',.2%',
    y2_series_axis_format=',.0',
    y1_series_title='Bitfinex Change in Balances vs. Volume (BTC)', data_source='Glassnode, CoinGecko')

In [273]:
chart_utils.two_axis_chart(
    bitfinex.loc[bitfinex['date_x'] >= '2020-01-01'], 'date_x',
    y1_series='bitfinex', y2_series='btc_volume_rolling_28d', y1_series_axis_format=',. BTC',
    y2_series_axis_format=',.0',
    title='Bitfinex BTC Balances vs. BTC Volume',
    y1_series_title='Bitfinex Balances (BTC)',
    data_source='Glassnode, CoinGecko')

In [129]:
bitfinex

,snapped_at,volume,date,volume_rolling_28d
0,2019-10-24 14:00:00 +0000,1.356168e+08,2019-10-24 14:00:00+00:00,NaN
1,2019-10-25 14:00:00 +0000,7.429731e+07,2019-10-25 14:00:00+00:00,NaN
2,2019-10-26 14:00:00 +0000,6.345130e+08,2019-10-26 14:00:00+00:00,NaN
3,2019-10-27 14:00:00 +0000,1.645554e+08,2019-10-27 14:00:00+00:00,NaN
4,2019-10-28 14:00:00 +0000,2.096511e+08,2019-10-28 14:00:00+00:00,NaN
...,...,...,...,...
361,2020-10-19 14:00:00 +0000,7.025872e+07,2020-10-19 14:00:00+00:00,1.167691e+08
362,2020-10-20 14:00:00 +0000,1.519661e+08,2020-10-20 14:00:00+00:00,1.170876e+08
363,2020-10-21 14:00:00 +0000,2.241203e+08,2020-10-21 14:00:00+00:00,1.210427e+08
364,2020-10-22 14:00:00 +0000,3.943222e+08,2020-10-22 14:00:00+00:00,1.290391e+08


In [239]:
bitmex = pd.read_csv("/Users//Downloads/bitmex-trading-volume-1-year.csv")

In [240]:
bitmex['snapped_at'] = [x[:10] for x in bitmex['snapped_at']]

In [241]:
bitmex['date'] = pd.to_datetime(bitmex['snapped_at'])

In [242]:
bitmex = get_extra_datetime_cols(bitmex, 'snapped_at', "%Y-%m-%d")
bitmex = bitmex.merge(cm, on='day_string')

In [245]:
bitmex = bitmex.merge(df, on='day_string')

In [246]:
bitmex

,snapped_at,volume,date_x,datetime_x,day_string,date_y,PriceUSD,datetime_y,btc_volume,btc_volume_rolling_28d,...,huobi_pct_change,huobi_net_change,kraken_pct_change,kraken_net_change,luno_pct_change,luno_net_change,okex_pct_change,okex_net_change,poloniex_pct_change,poloniex_net_change
0,2019-12-31,1.245818e+09,2019-12-31,2019-12-31,2019-12-31,2019-12-31,7167.395055,2019-12-31,173817.393338,264079.418543,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2020-01-01,1.145781e+09,2020-01-01,2020-01-01,2020-01-01,2020-01-01,7170.631869,2020-01-01,159788.010497,255743.610329,...,0.001631,571.674341,0.002137,314.938573,-0.000376,-6.663801,0.000498,85.465332,0.001249,77.742628
2,2020-01-02,1.185973e+09,2020-01-02,2020-01-02,2020-01-02,2020-01-02,6946.825269,2020-01-02,170721.566897,248169.115867,...,0.001345,471.209967,0.005905,870.245939,0.006511,115.272508,0.002404,412.888223,-0.000076,-4.724207
3,2020-01-03,3.299895e+09,2020-01-03,2020-01-03,2020-01-03,2020-01-03,7315.309360,2020-01-03,451094.433438,255478.205396,...,0.000031,10.868073,-0.000647,-95.359525,0.010981,194.407841,0.006601,1133.597803,-0.003355,-208.782431
4,2020-01-04,1.594468e+09,2020-01-04,2020-01-04,2020-01-04,2020-01-04,7343.164194,2020-01-04,217136.410384,254285.419032,...,-0.000250,-87.440866,0.005350,788.378727,0.007653,135.482359,0.011105,1906.934305,-0.005177,-322.175137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,2020-10-17,7.474363e+08,2020-10-17,2020-10-17,2020-10-17,2020-10-17,11362.802217,2020-10-17,65779.219575,129875.975892,...,-0.202790,-71060.527952,-0.001848,-272.332408,-0.020414,-361.395392,0.179636,30848.102688,-0.232803,-14488.989290
292,2020-10-18,7.903779e+08,2020-10-18,2020-10-18,2020-10-18,2020-10-18,11489.622565,2020-10-18,68790.585020,128296.349268,...,-0.211124,-73980.893632,-0.006009,-885.470988,-0.020990,-371.603041,0.180322,30965.885069,-0.233210,-14514.329331
293,2020-10-19,1.097708e+09,2020-10-19,2020-10-19,2020-10-19,2020-10-19,11745.811968,2020-10-19,93455.280908,123486.639811,...,-0.219215,-76816.065175,-0.006950,-1024.171391,-0.028143,-498.236985,0.180564,31007.359054,-0.234514,-14595.482142
294,2020-10-20,1.956148e+09,2020-10-20,2020-10-20,2020-10-20,2020-10-20,11929.457251,2020-10-20,163976.313561,123630.385267,...,-0.224529,-78677.952294,-0.003969,-584.848482,-0.038705,-685.218984,0.183602,31529.154241,-0.235667,-14667.264331


In [253]:
bitmex['btc_volume'] = bitmex['volume'] / bitmex['PriceUSD']
bitmex['btc_volume_rolling_28d'] = bitmex.btc_volume.rolling(28, min_periods=28).mean()
bitmex['btc_volume_rolling_7d'] = bitmex.btc_volume.rolling(7, min_periods=7).mean()

In [256]:
chart_utils.two_axis_chart(
    bitmex.loc[bitmex['date_x'] >= '2020-01-01'], 'date_x',
    y1_series='bitmex_pct_change', y2_series='btc_volume_rolling_28d', y1_series_axis_format=',.2%',
    y2_series_axis_format=',.0',
    y1_series_title='BitMex Change in Balances vs. Volume (BTC)', data_source='Glassnode, CoinGecko')

In [274]:
chart_utils.two_axis_chart(
    bitmex.loc[bitmex['date_x'] >= '2020-01-01'], 'date_x',
    y1_series='bitmex', y2_series='btc_volume_rolling_28d', y1_series_axis_format=',. BTC',
    y2_series_axis_format=',.0',
    title='BitMex BTC Balances vs. BTC Volume',
    y1_series_title='BitMex Balances (BTC)',
    data_source='Glassnode, CoinGecko')